### Learning Objectives
* To implement the basic operations on a doubly linked lists
* To distinguish stacks from queues
* To implement stacks and queues using doubly linked lists

### Instructions
Read and study the following sections, run their code examples and solve their challenges. This worksheet has the following challenges:
* [CHALLENGE 01](#ch01)
* [CHALLENGE 02](#ch02)
* [CHALLENGE 03](#ch03)

Run your coding challenges and fix any errors they might have before downloading and submitting your completed worksheet for grading. When done, open the menu **File >> Download as >> HTML (.html)** to download your worksheet in HTML format. **Submit the downloaded *.html* file via Canvas**.

# Doubly linked lists
Linked lists where each node has two pointers one to the node before it and the other to the node after it are called **doubly linked lists**. Such lists allow for iterating over nodes from `back` to `front`. 

Let's see what implementing a doubly linked list looks like compared to the previously implemented singly linked list.

## The `node` data structure
Let's start with the `Node` data structure.

In [ ]:
template<typename T>
struct Node {
  T info;
  Node<T>* next;
  Node<T>* prev;

  Node(T info): info(info), next(nullptr), prev(nullptr){}
  Node(T info, Node<T>* nxt, Node<T>* prv): info(info), next(nxt), prev(prv){}
};

Here a node consists of three pieces: `info` for storing the actual value of the node, `next` which is a pointer to the next node in the list, and `prev` which is a pointer to the previous node in the list. It's having this `prev` pointer that makes doubly linked lists different from singly linked lists. 

We now can use this `Node` structure to create the linked list class. 

## The `List` ADT

In [ ]:
#include <iostream>
#include <initializer_list>

template<typename T>
class List {
public:
  List(){};
  List(const std::initializer_list<T>& lst){
    for(auto it = lst.begin(); it != lst.end(); ++it){
      add_back(*it);
    }
  }
  List(const List<T>& lst){
    auto current = lst.front;
    while(current){
      add_back(current->info);
      current = current->next;
    }     
  }
  List<T>& operator=(const List<T>& lst){
    deleteAll();
    auto current = lst.front;
    while(current){
      add_back(current->info);
      current = current->next;
    }

    return *this;      
  }
    
  List<T>& add_back(T info);
  List<T>& add_front(T info);
  bool add_before(T info, T before);
    
  bool remove_back();
  bool remove_front();
  bool remove(T info);
    
  bool find(T info) {
    auto current = front;
    while(current){
      if(current->info == info){
        return true;
      }

      current = current->next;
    }

    return false;      
  }
    
  friend std::ostream& operator<<(std::ostream& out, const List<T>& lst){
    auto current = lst.front;
    out << "[" << lst.size << "]: ";
    while(current){
      out << current->info << " ";
      current = current->next;
    }

    return out;
  }
    
  unsigned getSize() const { return size; }
  bool empty() const { return size == 0u; }
  T peek_front() const { 
    if(empty()) throw std::runtime_error("List is empty");
    return front->info; 
  }
  T peek_back() const { 
    if(empty()) throw std::runtime_error("List is empty");
    return back->info; 
  }
  void clear() { deleteAll(); }
  ~List(){ deleteAll(); }
    
protected:
  unsigned size = 0u;
  Node<T>* front = nullptr;   
  Node<T>* back = nullptr;
    
  void deleteAll(){
    auto current = front;
    while(current){
      auto tmp = current;
      current = current->next;
      delete tmp;
    }

    front = nullptr;
    back = nullptr;
    size = 0u;
  }
};

This is the same class as the singly linked list we implemented before. The only difference is that when we implement the `add` and `remove` functions, we must set the `prev` pointers of the nodes properly as nodes are added to or removed from the list. 

## Adding nodes to a doubly linked list
There are different ways to add nodes to a linked list. Nodes can be added to the back of the list using the `add_back` function or at the front using the `add_front` function.

The `add_back` function must check if the list has a back node and, if so, makes the newly created node the new back, and let its `prev` pointer point to the old back. If not, the list is empty and the newly created node will become both the front and back of the list. It finally increments the size of the list by one.

In [ ]:
template<typename T>
List<T>& List<T>::add_back(T info){
  auto node = new Node<T>(info, nullptr, back);
  if(back){
    back->next = node;
    back = node;
  } else { // empty list
    front = node;
    back = node;
  }

  size++;
  return *this;
}

Similarly `add_front` needs to make the newly created node the new front of the list and increment the size of the list by one. The the `prev` pointer of the old front needs to point to the new front and the `prev` pointer of the new front must be `nullptr`.

In [ ]:
template<typename T>
List<T>& List<T>::add_front(T info){
  auto node = new Node<T>(info, front, nullptr);
  if(front){
    front->prev = node;
    front = node;
  } else { // empty list
    front = back = node;
  }

  size++;

  return *this;
}

## Removing nodes from a doubly linked list

Having the `prev` pointer eliminates the need to loop over the list to find the node that precedes what node that will be removed. This makes `remove_back` in doubly linked lists than in singly linked lists.

In [ ]:
template<typename T>
bool List<T>::remove_back(){
  if(front == back){
    if(!front){
      return false;
    }

    delete back;
    front = back = nullptr;
  } else{
    auto pred = back->prev;
    pred->next = nullptr;
    delete back;
    back = pred;
  }

  size--;
  return true;
}

Similarly `remove_front` needs to remove the front node and make its next node (if any) the new front.

In [ ]:
template<typename T>
bool List<T>::remove_front(){
  if(front == back){
    if(!front){
      return false;
    }

    delete front;
    front = back = nullptr;
  } else{
    auto tmp = front;
    front = front->next;
    front->prev = nullptr;
    delete tmp;
  }

  size--;
  return true;
}

Given a value, we can also remove the node that has that value from anywhere in the list. Again, no need to find the predecessor of the node to be removed; the `prev` pointer of that node points to it. Here is what the `remove` function looks like.

In [ ]:
template<typename T>
bool List<T>::remove(T info){
  if(front == nullptr){
    return false;
  }else if(front->info == info){
    auto tmp = front;
    front = front->next;
    front->prev = nullptr;
    delete tmp;
    size--;
    return true;
  } else {
    auto current = front->next;
    while(current){
      if(current->info == info){
        if(current == back){
          back = current->prev;
        } else {
          current->next->prev = current->prev;    
        }
        current->prev->next = current->next;
        delete current;
        size--;
        return true;
      }
      current = current->next;
    }

    return true;
  }
}

Having done that. Here is an example using this class.

In [ ]:
#include <iostream>
using namespace std;

List<int> lst;
lst.add_back(4).add_front(3).add_back(5).add_front(2).add_back(10);
cout << lst << endl;
lst.add_front(12).add_back(1);
cout << lst << endl;

lst.remove_back();
lst.remove_front();
lst.remove(15);
lst.remove(2);
cout << lst << endl;

if(lst.find(5)){
    cout << 5 << " was found" << endl;
}

### <a id="ch01">CHALLENGE 01</a>
Go back to the code cell where the template class `List` is defined and add the following public member function to it.

```c++
bool add_before(T info, T before);
```
You'll need to restart and rerun all the code cells after that.

In the code cell below, define the `add_before` function. This function should create a new node with the given `info` and add it to the list right before the node containing the given `before` value. It then increments the size by one and returns `true`. If no node with the given `before` value is found, then no node is added to the list and `false` is returned. Make sure the `prev` and `next` pointers of impacted nodes are set up properly.

Run your code and make sure it compiles without errors and warnings.

In [ ]:
// TODO: Implement add_before here

# Stacks
A stack is a simple data structure that allows us to put items on top of one another such that only the item at the top is accessible. Think of stacks of books or trays of dishes as example stacks. A stack works in **last in first out** (LIFO) manner. That is the last item to add to the stack is the first item to pop out.

A sack supports the following functions:
- `clear`: removes all items from the stack.
- `empty`: returns whether or not the stack is empty.
- `push`: adds an item to the top of the stack.
- `pop`: removes the item at the top of the stack.
- `top`: returns the item at the top of the stack without removing it.

There are many ways to implement a stack. We can use a dynamic array and keep expanding it as the stack grows. Alternatively, we can use a vector or a linked list for that. Let's use the above doubly linked list class `List`to implement it. And we can do that in one of two ways:
- By creating a stack class that inherits from the `List` class.
- By creating a stack class with a data member whose data type is the `List` class.

We'll use the first way to implement the stack, and we start by having the abstract class `StackADT` which captures what a stack is and what operations it supports.

In [ ]:
template <typename T>
class StackADT{
public:
  virtual void clear() = 0;
  virtual bool empty() const = 0;
  virtual void push(T e) = 0;
  virtual T top() const = 0;
  virtual T pop() = 0;
  virtual ~StackADT(){}
};

Here is a concrete stack class named `Stack` that inherits from both `StackADT` (publicly) and `List<>` (privately).

In [ ]:
template <typename T>
class Stack : public StackADT<T>, private List<T>{
public:
  Stack():List<T>(){}
  void clear() { List<T>::clear(); }
  bool empty() const { return this->size == 0u; }
  void push(T e) { this->add_front(e); }
  T pop() { 
    if(empty()) throw runtime_error("Stack is empty");
      
    T e = this->front->info;
    this->remove_front(); 
    return e;
  }
    
  T top() const { 
    if(empty()) throw runtime_error("Stack is empty");
    return this->front->info; 
  }  
};

As you can see, this implementation is clean and short. It uses private inheritance to make all the functions inherited from the class `List<T>` private to the `Stack` class and therefore cannot be passed down to any class inheriting from `Stack`. 

Here is an example using this `Stack` implementation.

In [ ]:
Stack<char> s;
s.push('H');
s.push('e');
s.push('l');
s.push('l');
s.push('o');

cout << s.pop() << endl;
cout << s.pop() << endl;
cout << s.pop() << endl;
cout << s.pop() << endl;
cout << s.pop() << endl;
cout << s.pop() << endl;

### <a id="ch02">CHALLENGE 02</a>
In the code cell below, create a class named `VectorizedStack` that uses a vector (from the `<vector>` header file) data member to implement the functions defined in `StackADT`. In the code cell after that, test your class by creating an object of it and calling its member functions.

In [ ]:
// TODO: VectorizedStack class goes here

In [ ]:
// TODO: Test your class here

## Queues
A queue is like a waiting line; it grows by adding items to its back and shrinks by removing items from its front. Unlike a stack, which is a last in first out (lIFO), a queue is a first in first out (FIFO) data structure. That is while a stack uses only one end to push items to and pop them from, a queue uses both ends: one for adding new items and one for removing them. That means the last item has to wait until all items preceding it on the queue are removed. 

A queue must support the following operations:
- `clear`: removes all items fron the queue.
- `empty`: returns whether or not the stack is empty.
- `enqueue`: adds an item at the end of the queue.
- `dequeue`: removes the first item from the queue.
- `peek_front`: returns the first item in the queue without removing it.

Like the stack, we can use the `List` class to implement a queue in one of two ways:
- By creating a queue class that inherits from `List` class.
- By creating a queue class with a data member whose data type is the `List` class.

This time we'll use the second way. Here is the abstract class `QueueADT` that captures what a queue is and what operations it support.

In [ ]:
template <typename T>
class QueueADT {
public:
  virtual void clear() = 0;
  virtual bool empty() const = 0;
  virtual void enqueue(T e) = 0;
  virtual void dequeue() = 0;
  virtual T peek_front() const = 0;
  virtual ~QueueADT(){}
};

And here is a concrete `Queue` class.

In [ ]:
template <typename T>
class Queue : public QueueADT<T>{
private:
  List<T> lst;
public:
  Queue():lst(List<T>()){}
  void clear() { lst.clear(); }
  bool empty() const { return lst.empty(); }
  void enqueue(T e) { lst.add_back(e); }
  void dequeue() { 
    if(!empty()){
        lst.remove_front();
    }
  }
    
  T peek_front() const { 
    if(empty()) throw runtime_error("Queue is empty");
    return lst.peek_front(); 
  }  

};

Here is an example using this `Queue` implementation.

In [ ]:
Queue<char> q;
q.enqueue('H');
q.enqueue('e');
q.enqueue('l');
q.enqueue('l');
q.enqueue('o');

cout << q.peek_front() << endl;
q.dequeue();
cout << q.peek_front() << endl;
q.dequeue();
cout << q.peek_front() << endl;
q.dequeue();
cout << q.peek_front() << endl;
q.dequeue();
cout << q.peek_front() << endl;
q.dequeue();
cout << q.peek_front() << endl;
q.dequeue();

### <a id="ch03">CHALLENGE 03</a>

Consider a stack $S$ and a queue $Q$ each with $n$ items. Answer the following running-time questions using the Big-O notation as a function of $n$.

**Q1**. What is the running time of each member function of the `Stack` class?

**Q2**. What is the running time of each member function of the `Queue` class?